In [54]:
import json
import pandas as pd
import re
import shutil
import os
import yaml

In [31]:
# load all parties name
df_zone_candidates = pd.read_csv('../data/candidates.csv')
df_pl_candidates = pd.read_csv('../data/partylist_candidates.csv')
party_names = list(map(lambda p: p.strip(), list(set(df_zone_candidates.PartyName.values))))

In [34]:
'We have %d zone candidates, %d party list, (%d in total)' % (
    len(df_zone_candidates), len(df_pl_candidates),
    len(df_zone_candidates) + len(df_pl_candidates)
)

'We have 11181 zone candidates, 2917 party list, (14098 in total)'

In [13]:
RAW_DATA_PATH = '/Users/heytitle/projects/politician-business/raw-data-before-verify/'
FINAL_DATA_PATH = '/Users/heytitle/projects/politician-business/app/assets/data'

In [32]:
df_zone_candidates['name'] = df_zone_candidates.apply(lambda p: '%s %s' % (p['FirstName'], p['LastName']), axis=1)
df_pl_candidates['name'] = df_pl_candidates.apply(lambda p: '%s %s' % (p['FirstName'], p['LastName']), axis=1)

In [35]:
nameToAttributes = dict(map(lambda x: (x['name'], x), 
                            df_zone_candidates.to_dict('record') +
                            df_pl_candidates.to_dict('record')
                           ))

In [36]:
nameToAttributes['ไกรสีห์ รัตนเศียร']

{'Unnamed: 0': 0,
 'CandidateId': '82547da2-ee9e-4e55-a9e0-9aa414956075',
 'CandidateNo': 1,
 'FirstName': 'ไกรสีห์',
 'LastName': 'รัตนเศียร',
 'PartyName': 'ชาติพัฒนา',
 'PartyNumber': 45,
 'Title': 'นาย',
 'Votable': True,
 'province_id': 23,
 'province_name': 'ตราด',
 'zone_number': 1,
 'name': 'ไกรสีห์ รัตนเศียร'}

In [ ]:
nameToAttributes = df_zone_candidates.to_di

In [93]:

def check_name(name):
    res = df_zone_candidates[df_zone_candidates.name == name]
    print(res.to_dict())
    
    res = df_pl_candidates[df_pl_candidates.name == name]
    print(res.to_dict())
    
check_name('จีรวัฒน์ อุดมสุด')

{'Unnamed: 0': {}, 'CandidateId': {}, 'CandidateNo': {}, 'FirstName': {}, 'LastName': {}, 'PartyName': {}, 'PartyNumber': {}, 'Title': {}, 'Votable': {}, 'province_id': {}, 'province_name': {}, 'zone_number': {}, 'name': {}}
{'Unnamed: 0': {1177: 1327}, 'CandidateId': {1177: 'e3262ce8-6833-406d-a226-e57da8970242'}, 'CandidateNo': {1177: 49}, 'FirstName': {1177: 'จีรวัฒน์'}, 'LastName': {1177: 'อุดมสุด'}, 'PartyName': {1177: 'เสรีรวมไทย'}, 'PartyNumber': {1177: 145}, 'Title': {1177: 'พันตำรวจตรี'}, 'Votable': {1177: True}, 'name': {1177: 'จีรวัฒน์ อุดมสุด'}}


# Move data files to temp

In [26]:
# for p in party_names:
#     with open('%s/%s.json' % (DATA_PATH, p)) as f:
#         data = json.load(f)
#         with open('%s/bak/%s.json' % (DATA_PATH, p), 'w') as outfile:
#             outfile.write(json.dumps(data, indent=4, sort_keys=True, ensure_ascii=False))
#             outfile.close()
# # remove name prefix

# # party_names

In [ ]:

if os.path.isdir(FINAL_DATA_PATH):
    shutil.rmtree(FINAL_DATA_PATH)
    
os.mkdir(FINAL_DATA_PATH)


with open("/Users/heytitle/projects/politician-business/name-mapping.yaml", "r") as f:
    nameMapping = yaml.load(f, Loader=yaml.FullLoader)
    
credenToEDTMapping = nameMapping['credenToedt']


partyIdx = 0
steps = 10
str_idx = partyIdx*steps
stp_idx = (partyIdx+1)*steps
print(str_idx, stp_idx)
# for i, p in enumerate(party_names[str_idx:stp_idx]):
for i, p in enumerate(party_names):
    with open('%s/%s.json' % (RAW_DATA_PATH, p)) as f:
        candidates = json.load(f)
        print('# %2d. %s ====' % (i+1, p))
        for c in candidates:
            name = c['name']

            nameWithPrefix = '%s%s' % (nameToAttributes[name]['Title'], name)
            
            newRelatedTo = []
            if c['relatedTo']:
                print('> %s' % nameWithPrefix)
            for o in c['relatedTo']:
                pName_before_mapping = o['PARTNER'].strip()
                
                if pName_before_mapping in credenToEDTMapping.keys():
                    pName = credenToEDTMapping[pName_before_mapping]
                    print('===> Manual override name from Creden.co (|%s:->:%s|)' % (pName_before_mapping, pName))
                else:
                    pName = pName_before_mapping
                    
                result = re.search(r"^%s$" % nameWithPrefix.replace('*', ''), pName.replace('*', ''))
                if not result:
                    print('|%s:>>:%s|\t\tYAML>>%s: %s<<'%(nameWithPrefix, pName, pName, nameWithPrefix))
                else:
                    newRelatedTo.append(o)
            diff = len(c['relatedTo'])-len(newRelatedTo)

            if diff > 0:
                print(' -- filter %d companies from %s (from %d to %d)' %
                      (diff, name, len(c['relatedTo']), len(newRelatedTo))
                     )



            c['relatedTo'] = newRelatedTo
        
        with open('%s/%s.json' % (FINAL_DATA_PATH, p), 'w') as outfile:
            outfile.write(json.dumps(candidates, indent=4, sort_keys=True, ensure_ascii=False))
            outfile.close()
        print('')

0 10
#  1. พลังศรัทธา ====
> นายบุญนำ วังตะเคน
> นายมนูญ เรืองสุวรรณ์
> นายสมศักดิ์ สงยอด
> นายนนทชัย ศิริผลา
> นางสาวสุทาทิพย์ โคตาสี
> นายจุมพล รัตนวงศ์
> นายมงคล สงพระ
> นายสว่าง สาธร
> นายธนาคม สังข์คร
> นายวรเทพ กระจ่างโพธิ์
> นายสันติ สีโมรส
> พันเอกสันติ โอเงิน
===> Manual override name from Creden.co (|พันโทสันติ โอเงิน:->:พันเอกสันติ โอเงิน|)
===> Manual override name from Creden.co (|นายสันติ โอเงิน:->:พันเอกสันติ โอเงิน|)
===> Manual override name from Creden.co (|นายสันติ โอเงิน:->:พันเอกสันติ โอเงิน|)
===> Manual override name from Creden.co (|นายสันติ โอเงิน:->:พันเอกสันติ โอเงิน|)
===> Manual override name from Creden.co (|นายสันติ โอเงิน:->:พันเอกสันติ โอเงิน|)
===> Manual override name from Creden.co (|นายสันติ โอเงิน:->:พันเอกสันติ โอเงิน|)
> นางสาวพิศมัย เสนาวงค์
> นายโสภณ ผิวขำ

#  2. พลังไทยดี ====
> นายสหาย นิลกำแหง
> นายสาธุ อนุโมทามิ
> นายสำเภา บุญเปี่ยม

#  3. ชาติไทยพัฒนา ====
> นายสมบูรณ์ ไพรวัลย์
> นายชารินทร์ สิงห์ดี
> นายณรงค์ สิงห์ฆาฬะ
> นายจันทรัสม์ ศรีส

|นายมนตรี จันทร:>>:นายมนตรี จันทร์แดง|		YAML>>นายมนตรี จันทร์แดง: นายมนตรี จันทร<<
|นายมนตรี จันทร:>>:นายมนตรี จันทร์ที|		YAML>>นายมนตรี จันทร์ที: นายมนตรี จันทร<<
|นายมนตรี จันทร:>>:นายมนตรี จันทรกุลพงศา|		YAML>>นายมนตรี จันทรกุลพงศา: นายมนตรี จันทร<<
|นายมนตรี จันทร:>>:นายมนตรี จันทร์นิยม|		YAML>>นายมนตรี จันทร์นิยม: นายมนตรี จันทร<<
|นายมนตรี จันทร:>>:นายมนตรี จันทร์แจ่มใส|		YAML>>นายมนตรี จันทร์แจ่มใส: นายมนตรี จันทร<<
|นายมนตรี จันทร:>>:นายมนตรี จันทร์เรือง|		YAML>>นายมนตรี จันทร์เรือง: นายมนตรี จันทร<<
|นายมนตรี จันทร:>>:นายมนตรี จันทร์สกุล|		YAML>>นายมนตรี จันทร์สกุล: นายมนตรี จันทร<<
|นายมนตรี จันทร:>>:นายมนตรี จันทร์เรือง|		YAML>>นายมนตรี จันทร์เรือง: นายมนตรี จันทร<<
|นายมนตรี จันทร:>>:นายมนตรี จันทร์โชติเสถียร|		YAML>>นายมนตรี จันทร์โชติเสถียร: นายมนตรี จันทร<<
|นายมนตรี จันทร:>>:นายมนตรี จันทรัตน์เจริญ|		YAML>>นายมนตรี จันทรัตน์เจริญ: นายมนตรี จันทร<<
|นายมนตรี จันทร:>>:นายมนตรี จันทร์พา|		YAML>>นายมนตรี จันทร์พา: นายมนตรี จันทร<<
|นายมนตรี จันทร:>>:นายมนตรี จันทรกุลพงศา|		

|นางสาวจารุวรรณ วงศ์พิมล:>>:นางสาวจารุวรรณ วงศ์พิมลพร|		YAML>>นางสาวจารุวรรณ วงศ์พิมลพร: นางสาวจารุวรรณ วงศ์พิมล<<
 -- filter 3 companies from จารุวรรณ วงศ์พิมล (from 3 to 0)
> นางสาวเนตรทราย วอนเมือง
> นางศิริพร นันตา
|นางศิริพร นันตา:>>:นางสาวศิริพร นันตาวงศ์|		YAML>>นางสาวศิริพร นันตาวงศ์: นางศิริพร นันตา<<
|นางศิริพร นันตา:>>:นางสาวศิริพร นันตาวรัตน์|		YAML>>นางสาวศิริพร นันตาวรัตน์: นางศิริพร นันตา<<
|นางศิริพร นันตา:>>:นางสาวศิริพร นันตาวรัตน์|		YAML>>นางสาวศิริพร นันตาวรัตน์: นางศิริพร นันตา<<
 -- filter 3 companies from ศิริพร นันตา (from 3 to 0)
> นายสมพร จินดา
|นายสมพร จินดา:>>:นางสมพร จินดากาศ|		YAML>>นางสมพร จินดากาศ: นายสมพร จินดา<<
|นายสมพร จินดา:>>:นายสมพร จินดาศิริพันธ์|		YAML>>นายสมพร จินดาศิริพันธ์: นายสมพร จินดา<<
|นายสมพร จินดา:>>:นายสมพร จินดาศิริพันธ์|		YAML>>นายสมพร จินดาศิริพันธ์: นายสมพร จินดา<<
 -- filter 3 companies from สมพร จินดา (from 3 to 0)
> นางสาวธนวรรณ เกษมสุข
> นางสาวไอยพราว พัชร์ปวริศร์
> นายภูเบศร์ มงคลวัฒน์
> นางสาวภิญญาภรณ์ คงพิทักษ์
> นายฟ้าใส บ

> นายสมศักดิ์ ราชเจริญ
> นายไพฑูรย์ หอมกลุ่น
> นายกรกช บุตรสิม
> ร้อยตรีไพศาล เชิดชู
|ร้อยตรีไพศาล เชิดชู:>>:นายไพศาล เชิดชูบัณฑิต|		YAML>>นายไพศาล เชิดชูบัณฑิต: ร้อยตรีไพศาล เชิดชู<<
===> Manual override name from Creden.co (|นายไพศาล เชิดชู:->:ร้อยตรีไพศาล เชิดชู|)
|ร้อยตรีไพศาล เชิดชู:>>:นายไพศาล เชิดชูชาติ|		YAML>>นายไพศาล เชิดชูชาติ: ร้อยตรีไพศาล เชิดชู<<
 -- filter 2 companies from ไพศาล เชิดชู (from 3 to 1)
> ร้อยตำรวจตรีสมพล เกษมมาลา
===> Manual override name from Creden.co (|นายสมพล เกษมมาลา:->:ร้อยตำรวจตรีสมพล เกษมมาลา|)
> นายสะอาด วรนาม
> นายโอภาส อาลมิสรี
> นายสงกรานต์ สมัครเขตรการณ์
> พันตำรวจตรีจีรวัฒน์ อุดมสุด
===> Manual override name from Creden.co (|พลตำรวจตรีจีรวัฒน์ อุดมสุด:->:พันตำรวจตรีจีรวัฒน์ อุดมสุด|)
===> Manual override name from Creden.co (|พลตำรวจตรีจีรวัฒน์ อุดมสุด:->:พันตำรวจตรีจีรวัฒน์ อุดมสุด|)
> นายสุบรรณ มหาชนนท์
> นายชาญชาติ มูลดี
> นายสมาน เจริญสูงเนิน
> นายสวาท สุทธิอาคาร
> นายกมลภู สวัสดิสาร
> นาวาอากาศเอกประสิทธิ์ บุญประเสริฐ
===> Manual override

> นายประโชติ สิริวัฒน์
> นางสาวรักษ์หฤทัย ยกสุขฤทัยไขข่าว
> นางสาวลักษณีรายา คณานุรักษ์
===> Manual override name from Creden.co (|นางลักษณีรายา คณานุรักษ์ ณ สงขลา:->:นางสาวลักษณีรายา คณานุรักษ์|)
===> Manual override name from Creden.co (|นางลักษณีรายา คณานุรักษ์:->:นางสาวลักษณีรายา คณานุรักษ์|)
===> Manual override name from Creden.co (|นางลักษณีรายา คณานุรักษ์:->:นางสาวลักษณีรายา คณานุรักษ์|)
===> Manual override name from Creden.co (|นางลักษณีรายา คณานุรักษ์:->:นางสาวลักษณีรายา คณานุรักษ์|)
> นายวัชระ สนธิชัย
> นายสงคราม เมืองมนตรี
> นายสมพจน์ โรหิตาภิรมย์
> นายสล้าง ชินะกานนท์
> นายสุรัตน์ พิมนิสัย
> นางอารีรัตน์ เลาหพล
> นายสุธรรม ลิ้มสุวรรณเกษม
> นายมนัส เพ็ญนุกูล
> นายวิจารณ์ จงภักดี

# 46. ภูมิพลังเกษตรกรไทย ====
> นายบุญเลิศ แก้วอินทร์
> นายดอน ศรีนวลขาว
> นางสาวจุฑามาศ จันทรสมบัติ
> นางกริตยา แสงสุริยันต์
===> Manual override name from Creden.co (|นางสาวกริตยา แสงสุริยันต์:->:นางกริตยา แสงสุริยันต์|)
> นายประพันธ์ มีภพ
> นายพิศณุพงศ์ สิทธิโชคแก้วมูล
> นายประเสริฐ เผ่าพันธ์
>

> นายบุญเหนือ พุทธบุรี
> นายปฐวี ชัยวัฒน์
> นายสังวน เจิมแป้น
> นายธรรมศักดิ์ เติมจิตรอารีย์
> นายสุรินทร์ งามขำ
> นายวิภาค ตันติกิจชาญชัย
> พลตำรวจเอกยงยุทธ เทพจำนงค์
> นางสาวจุฬาพร ทับทิมธงไชย
> พลตำรวจตรีอภิรักษ์ หงษ์ทอง
===> Manual override name from Creden.co (|นายอภิรักษ์ หงษ์ทอง:->:พลตำรวจตรีอภิรักษ์ หงษ์ทอง|)
> พลตำรวจตรีปิยะ สุขประเสริฐ
===> Manual override name from Creden.co (|นายปิยะ สุขประเสริฐ:->:พลตำรวจตรีปิยะ สุขประเสริฐ|)
===> Manual override name from Creden.co (|นายปิยะ สุขประเสริฐ:->:พลตำรวจตรีปิยะ สุขประเสริฐ|)
> พลตำรวจตรีสุรศักดิ์ บุญกลาง
===> Manual override name from Creden.co (|นายสุรศักดิ์ บุญกลาง:->:พลตำรวจตรีสุรศักดิ์ บุญกลาง|)
> นางสาวชนากานต์ ทับทิมธงไชย
> นางสาวภิญญดา ไพรัช
> นางสาวสรณพรรณ พัวเศรษฐิกุล
> นายสิทธิชัย จันทร์ศุภวิบูลย์
> นางสาววรัญญา คำปวน
> ร้อยตำรวจตรีประจักษ์ กางพันเทศ
===> Manual override name from Creden.co (|นายประจักษ์ กางพันเทศ:->:ร้อยตำรวจตรีประจักษ์ กางพันเทศ|)
> นายสมบูรณ์ แสงทองสุข
> นายทองดี แสงจันทร์
> นางสาวชนนิกานต์ พันธ์ถนอ